# FUSIONAR DATA RECOPILADA: MD + SOFAESCORE 

In [2]:
import pandas as pd
import openpyxl
import os 
from unidecode import unidecode
import Levenshtein

In [16]:
import pandas as pd

jornadaS = 'jornada_12'
jornadaF = 'J12'
partido ="performance_jugadores_partido_Real_Sociedad_Barcelona"

# Rutas de los archivos Excel
excel1_path = f'../Sofascore-data-collector/performance_jugadores_por_partidos/{jornadaS}/xlsx/{partido}.xlsx'
excel2_path = f'../Mister-Fantasy-Mundo-Deportivo-Scraper/output_scraper/{jornadaF}.xlsx'

# Leer los datos de los archivos Excel
df1 = pd.read_excel(excel1_path, header=None)
df2 = pd.read_excel(excel2_path, header=None)

# Obtener todas las celdas de la fila 1 (que ahora es la segunda fila después de desactivar el encabezado)
fila_excel1 = df1.iloc[0, :].dropna().tolist()
fila_excel2 = df2.iloc[0, :].dropna().tolist()

# Concatenar las listas
fila_concatenada =  fila_excel2 + fila_excel1

# Crear un DataFrame de pandas con una sola fila y múltiples columnas
df = pd.DataFrame([fila_concatenada])

# Escribir el DataFrame en un archivo Excel
df.to_excel(f'../data/{partido}.xlsx', index=False, header=False)

###############################################################################################
# Inicializar el conjunto de valores encontrados
valores_encontrados = set()
df_fusionado = pd.DataFrame()
contador_coincidencias=0
contador_global=0
value_to_compare1=None
value_to_compare2=None

# Iterar sobre las filas de df1 y comparar con las filas de df2
for index_df1, row_df1 in df1.iterrows():
    value_to_compare1 = row_df1.iloc[0] 
    value_to_compare1 =value_to_compare1.lower()
    value_to_compare1 =unidecode(value_to_compare1)
    value_to_compare1 = unidecode(value_to_compare1)
    value_to_compare1=value_to_compare1.replace(" ", "")
    
    coincidencia_encontrada = False
    contador_global+=1
    # Iterar sobre las filas de df2
    for index_df2, row_df2 in df2.iterrows():
        #print("-----",value_to_compare1,"-----",value_to_compare2,"-----")
        value_to_compare2 = row_df2.iloc[0]
        value_to_compare2 =value_to_compare2.lower()
        value_to_compare2 =unidecode(value_to_compare2)
        value_to_compare2 = unidecode(value_to_compare2)
        value_to_compare2=value_to_compare2.replace(" ", "")
        
        # Calcular la distancia de Levenshtein
        distancia = Levenshtein.distance(value_to_compare1, value_to_compare2)
        # Establecer un umbral para considerar coincidencias
        umbral = 2  


        if distancia <= umbral: 
            print(f"Coincidencia encontrada: excell1-fila-{index_df1} <-> excell2-fila.{index_df2} , {value_to_compare1} == {value_to_compare2}")
            valores_encontrados.add(value_to_compare1) 
            
            # Obtener las listas de las filas
            fila_excel1 = df1.iloc[index_df1, :].tolist()
            fila_excel2 = df2.iloc[index_df2, :].tolist()

            # Concatenar las listas
            fila_concatenada = fila_excel2 + fila_excel1

            # Crear un DataFrame de pandas con una sola fila y múltiples columnas
            df_nueva_fila = pd.DataFrame([fila_concatenada])

            # Leer el archivo Excel existente
            df_existente = pd.read_excel(f'../data/{partido}.xlsx', header=None)

            # Concatenar el DataFrame existente con la nueva fila
            df_final = pd.concat([df_existente, df_nueva_fila], ignore_index=True)

            # Escribir el DataFrame final en el archivo Excel
            df_final.to_excel(f'../data/{partido}.xlsx', index=False, header=False)
            
            contador_coincidencias +=1
            coincidencia_encontrada = True

            
    # Imprimir si no se encontró ninguna coincidencia
    if not coincidencia_encontrada:
        print("-------------------------------------------------------------------------------")
        print(f"Coincidencia NO encontrada: excell1-fila-{index_df1} en {value_to_compare1}")
        print("-------------------------------------------------------------------------------")

print(" ")
print("__________________________________________________________________")
print("Total coincidencias: ",contador_coincidencias,"/",contador_global)
print("Precisión: ", (contador_coincidencias/contador_global)*100," %")
print("Done")

-------------------------------------------------------------------------------
Coincidencia NO encontrada: excell1-fila-0 en nombre
-------------------------------------------------------------------------------
Coincidencia encontrada: excell1-fila-1 <-> excell2-fila.23 , alexremiro == alexremiro
Coincidencia encontrada: excell1-fila-2 <-> excell2-fila.98 , hamaritraore == hamaritraore
Coincidencia encontrada: excell1-fila-3 <-> excell2-fila.121 , igorzubeldia == igorzubeldia
Coincidencia encontrada: excell1-fila-4 <-> excell2-fila.143 , robinlenormand == robinlenormand
Coincidencia encontrada: excell1-fila-5 <-> excell2-fila.136 , aihenmunoz == aihenmunoz
Coincidencia encontrada: excell1-fila-6 <-> excell2-fila.16 , braismendez == braismendez
Coincidencia encontrada: excell1-fila-7 <-> excell2-fila.142 , mikelmerino == mikelmerino
Coincidencia encontrada: excell1-fila-8 <-> excell2-fila.22 , martinzubimendi == martinzubimendi
Coincidencia encontrada: excell1-fila-9 <-> excell2-fila.